# Canopy LW + SW net Energy Balance test per layer

In [ ]:

# Use Plots:
using Plots
using BenchmarkTools
using Statistics
using Parameters

In [ ]:
using Revise
using Land
using Land.CanopyRT
using Land.Photosynthesis

In [ ]:

# Create an array of standard leaves (needs to be in Module later on:
arrayOfLeaves = Array{leafbio{FT,length(wl), length(wle), length(wlf),length(wle)*length(wlf)}, 1}(undef, CanopyRT.canopy.nlayers)
for i = 1:CanopyRT.canopy.nlayers
    #@show i
    arrayOfLeaves[i] = leafbio{FT, length(wl), length(wle), length(wlf),length(wle)*length(wlf)}()
    CanopyRT.fluspect!(arrayOfLeaves[i], CanopyRT.optis)
end


In [ ]:
# 4 Different steps to compute Short-Wave RT
@time CanopyRT.computeCanopyGeomProps!(canopy, angles,canOpt)
@time CanopyRT.computeCanopyMatrices!(arrayOfLeaves,canOpt)
@time CanopyRT.RTM_SW!(canopy, canOpt, canRad,sunRad, soil)
@time CanopyRT.deriveCanopyFluxes!(canopy, canOpt, canRad,sunRad, soil, arrayOfLeaves)
# Compute Long Wave (Last term is LW incoming in W m^-2)
@time CanopyRT.computeThermalFluxes!(arrayOfLeaves, canOpt, canRad, canopy, soil, [Float32(400.0)])

In [ ]:
#@show arrayOfLeaves[1].kChlrel
# Layer Temperatures are here:
@show canRad.T_sun;
@show canRad.T_shade;

In [ ]:
# Net Energy fluxes
@show canRad.intNetLW_shade;
@show canRad.intNetLW_sunlit;
@show canRad.intNetSW_shade;
@show canRad.intNetSW_sunlit;
@show canRad.RnSoilLW;
@show canRad.RnSoil;
#@show 1e6 * canRad.absPAR_sunCab;
@show 1e6 * canRad.absPAR_shadeCab;

In [ ]:
plot(wl, canRad.netSW_sunlit)

In [ ]:
iLAI = canopy.LAI/canopy.nlayers
plot(wl,1 ./canOpt.Ps[10] / iLAI * canRad.netSW_sunlit[:,10])
plot!(wl, sunRad.E_direct)

In [ ]:

l = leaf_params{Float32}();
l2 = leaf_params{Float32}();
# Create a Flux structure
met = meteo{Float32}();
l.Vcmax25=120
l.Jmax25=l.Vcmax25*1.8

In [ ]:
# initialize some reasonable values

met.e_air=500;
met.T_air=293;
l.APAR = 1.0e6 * canRad.absPAR_shadeCab[1];
1e6*mean(canRad.absPAR_sunCab[:,:,1])

In [ ]:
mods = Photosynthesis.PhotoMods()

In [ ]:
l.dynamic_state = false
@btime Photosynthesis.CanopyPhotosynthesis!(mods, l, met, canRad)

In [ ]:
using Plots

plot(Float32(1.0e6) * canRad.absPAR_sunCab[:,:,1][:], canRad.GPP_sun[:,:,1][:], seriestype = :scatter,legend=:bottomright, label="Individual sunlit leaves in one layer  (A(APAR))")
#plot!([Float32(1.0e6) * mean(canRad.absPAR_sunCab[:,:,1])], [f.an], seriestype = :scatter,  label="A(average(APAR))")
plot!([Float32(1.0e6) * mean(canRad.absPAR_sunCab[:,:,1])], [mean(canRad.GPP_sun)], seriestype = :scatter, label="average(A((APAR)))")
plot!(Float32(1.0e6) * canRad.absPAR_shadeCab[:], canRad.GPP_shade, seriestype = :scatter,legend=:bottomright, label="Individual shaded layers")



In [ ]:
plot(Float32(1.0e6) * canRad.absPAR_sunCab[:,:,1][:], canRad.ϕ_sun[:,:,1][:], seriestype = :scatter,legend=:bottomright, label="Individual sunlit leaves in one layer  (A(APAR))")

In [ ]:
sF = (canOpt.Ps[2:end]+canOpt.Ps[1:end-1])/2
plot((1 .- sF).*canRad.GPP_shade,reverse(layers), label="Shade" )
plot!((sF).*mean(canRad.GPP_sun,dims=(1,2))[1,1,:],reverse(layers),label="Sun"   )

In [ ]:
mean(canRad.GPP_sun,dims=(1,2))[1,1,:]

In [ ]:
levels = collect(0:canopy.LAI/canopy.nlayers:canopy.LAI)

In [ ]:
layers = (levels[2:end]+levels[1:end-1])/2

In [ ]:
plot((1 .- sF).*canRad.H_shade,reverse(layers), label="Shade" )
plot!((sF).*mean(canRad.H_sun,dims=(1,2))[1,1,:],reverse(layers),label="Sun"   )

In [ ]:
plot((1 .- sF).*canRad.LE_shade,reverse(layers), label="Shade" )
plot!((sF).*mean(canRad.LE_sun,dims=(1,2))[1,1,:],reverse(layers),label="Sun"   )

In [ ]:
l.LE

In [ ]:
met.T_air

In [ ]:
l.T

In [ ]:
l.esat-met.e_air

In [ ]:
l.esat

In [ ]:
met.e_air

In [ ]:
met.g_m_s_to_mol_m2_s/l.gs